## LAVA JATO - AN INFOGRAPHIC OF THE CORRUPTION SCANDALS IN BRAZIL

In [1]:
from os import listdir
from os.path import isfile, join

delacoes = ['./Delações/' + f for f in listdir('./Delações/') if isfile(join('./Delações/', f))]

delacoes[:10]



['./Delações/Colaboração premiada - Alberto Youssef - Depoimento 1.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 10.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 11.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 12.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 13.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 14.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 15.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 16.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 17.pdf',
 './Delações/Colaboração premiada - Alberto Youssef - Depoimento 18.pdf']

In [30]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfparser import PDFPage
from pdfminer.pdfparser import *
from io import StringIO

txtDelacoes = []

def convert_pdf_to_txt(path):
    retstr = StringIO()
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, retstr, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    fp = open(delacoes[0], 'rb')

    parser = PDFParser(fp)
    document = PDFDocument()
    parser.set_document(document)
    document.set_parser(parser)

    for page in document.get_pages():
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

txtDelacoes.append(convert_pdf_to_txt(delacoes[0]))

print(txtDelacoes[0][:300])

--------------------------------~-~ -

CONFIDENCIAL 
POLICIA FEDERAL 

SUPERINTEND~NCIA REGIONAL NO ESTAOO 00 PARANÁ 

TERMO DE COLABORAÇÃO N ° 01 

TERMO DE DECLARA Ç ÕES 

que presta ALBERTO YOUSSEF 

DRCOR - Delegacia Regional de Combate ao Crime Organizado 
DELEFIN - Delegacia de Repressão a C r


In [126]:
import re

def fixOcr(pattern):
    result = ""
    for c in pattern:
        result += c + "\s?"
        
    result = re.sub('[DO0]','[DO0]',result)
    result = re.sub('Ê','[Ê~]',result)
    result = re.sub('Á','[AÁ]',result)
    return result

def extractDelation(delationPdfText):
    delationPdfText = delationPdfText[re.search('\nque presta ',delationPdfText).span()[0]:]
    delationPdfText = re.sub(fixOcr('CONFIDENCIAL \nPOLICIA FEDERAL'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('\nCONFIDENCIAL'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('\nSUPERINTENDIONCIA'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('SUPERINTENDÊNCIA REGIONAL NO ESTAOO DO PARANÁ'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('REGIONAL NO ESTAOO DO PARANÁ'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('TERMO DE COLABORAÇÃO ') + 'N[^\n]+','',delationPdfText)
    delationPdfText = re.sub(fixOcr('TERMO DE DECLARAÇÕES '),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('DRCOR - Delegacia Regional de Comba') + '[tl]' + fixOcr('e ao Crime Organizado'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('TERMO DE DECLARAÇÕES'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('DELEFIN - Delegacia de Repressão a Crimes contra o Sistema Financeiro e Desvio de Verbas Públicas'),'',delationPdfText)
    delationPdfText = re.sub(fixOcr('DELEFIN - Delegacia de Repressão a Crimes contra o Sistema Financeiro e Desvio de Veribas Públicas'),'',delationPdfText)
    delationPdfText = delationPdfText[:re.search('\nDECLARANTE:',delationPdfText).span()[0]]
    delationPdfText = re.sub('\n',' ',delationPdfText)
    return delationPdfText
    
result = extractDelation(txtDelacoes[0])

words = str.split(result, ' ')

wordCount = {}

for word in words:
    if(re.match('\d', word) == None and len(word) > 1):
        fixedWord = str.lower(re.sub('[,;\."]$','',word))
        fixedWord = re.sub('^"','',fixedWord)
        if(fixedWord in wordCount):
            wordCount[fixedWord] += 1
        else:
            wordCount[fixedWord] = 1

mostCommonUselessWords = ['-', '', 'de','era', 'eram','junto','a', 'é','•', 'feito','que','diz','anteriormente','serem',
                          'fazer','tendo','quem','dos', 'segundo','ter','sendo','art.', 'conforme','qual','afirma','o', 
                          'pelas','podendo','quais','do', 'acerca','da', 'fim','no', 'em', 'se', 'como', 'um', 'uma', 
                          'os', 'as', 'para', 'e', 'por', 'na', 'com', 'Dos', 'ou', 'das', 'são', 'ao', 'entre', 'mais', 
                          'nos', 'ser', 'sua', 'mas', 'tem', 'também', 'pela', 'sobre', 'seu', 'pelo', 'pode', 'termos',
                          'assim', 'esta', 'nas', 'já', 'aos', 'questão', 'diferentes', 'termo', 'in', 'mesmo', 'século',
                          'outro', 'este', 'isso', 'pois', 'há', 'isto', 'lugar', 'the', 'esse', 'outros', 'sem', 'uso', 
                          'essa', 'lá', 'foi', 'ainda', 'ele', 'não', 'só', 'realização','determinada','algum','seria',
                          'assevera','pessoa','teve','próprio','ate','inicialmente','sempre','vezes','havia','pet','passou',
                          'presente','algumas','\x0c', 'período','recorda-se','antes','nota','prévio','seja','sido',
                          'todavia','feitos','inclusive','meio','teve','sido','declarante']

for uselessWord in mostCommonUselessWords:
    wordCount.pop(uselessWord, None)
    
sortedWordCount = sorted(wordCount.items(), key=lambda word: word[1])
sortedWordCount.reverse()
sortedWordCount[:100]

[('valores', 21),
 ('janene', 20),
 ('empreiteiras', 20),
 ('empresas', 20),
 ('paulo', 20),
 ('roberto', 17),
 ('contratos', 11),
 ('petrobras', 10),
 ('costa', 10),
 ('jose', 10),
 ('caso', 10),
 ('empresa', 10),
 ('questionado', 9),
 ('ano', 9),
 ('diretoria', 8),
 ('pagamentos', 7),
 ('esquema', 7),
 ('valor', 7),
 ('federal', 6),
 ('lei', 6),
 ('emissão', 6),
 ('relação', 6),
 ('grandes', 5),
 ('serviços', 5),
 ('espécie', 5),
 ('contratação', 5),
 ('pagos', 5),
 ('colaboração', 5),
 ('regra', 5),
 ('sabe', 5),
 ('youssef', 5),
 ('comissões', 5),
 ('art', 5),
 ('informações', 4),
 ('campanha', 4),
 ('procurador', 4),
 ('comissão', 4),
 ('custos', 4),
 ('organização', 4),
 ('posteriormente', 4),
 ('preso', 4),
 ('pp', 4),
 ('conta', 4),
 ('parte', 4),
 ('partido', 4),
 ('demais', 4),
 ('aditivos', 4),
 ('margem', 4),
 ('claudio', 4),
 ('criminosa', 4),
 ('notas', 4),
 ('ato', 4),
 ('dentre', 4),
 ('financeiro', 4),
 ('soube', 4),
 ('galvao', 4),
 ('orçamento', 3),
 ('pmdb', 3),
 ('

In [35]:
def getRegexPatternWithIntersectingSpaces(pattern):
    result = ""
    for c in pattern:
        result += c + "\s?"
    return result
        
getRegexPatternWithIntersectingSpaces("potato!!!")

'p\\s?o\\s?t\\s?a\\s?t\\s?o\\s?!\\s?!\\s?!\\s?'

In [47]:
fixOcr('SUPERINTENDÊNCIA REGIONAL NO ESTAOO DO PARANÁ')

'S\\s?U\\s?P\\s?E\\s?R\\s?I\\s?N\\s?T\\s?E\\s?N\\s?[DO0]\\s?[Ê~]\\s?N\\s?C\\s?I\\s?A\\s? \\s?R\\s?E\\s?G\\s?I\\s?[DO0]\\s?N\\s?A\\s?L\\s? \\s?N\\s?[DO0]\\s? \\s?E\\s?S\\s?T\\s?A\\s?[DO0]\\s?[DO0]\\s? \\s?[DO0]\\s?[DO0]\\s? \\s?P\\s?A\\s?R\\s?A\\s?N\\s?Á\\s?'

In [51]:
a = re.matches(fixOcr('SUPERINTENDÊNCIA REGIONAL NO ESTAOO DO PARANA'), 'SUPERINTENO~NCIA REGIONAL NO ESTADO DO PARANA')

print(a)

AttributeError: module 're' has no attribute 'matches'

In [74]:
print(re.search('\nDECLARANTE:', 'abc\nDECLARANTE:ABC').span()[0])

3


In [106]:
a = re.match('\d', '123')
print(a)

<_sre.SRE_Match object; span=(0, 1), match='1'>


In [103]:
help(re.match)

Help on function match in module re:

match(pattern, string, flags=0)
    Try to apply the pattern at the start of the string, returning
    a match object, or None if no match was found.

